In [ ]:
#upload workday's My_Recruiting_Jobs_-_More_Details report and transform

#import packages
import pandas as pd
import os
import io
import numpy as np

In [ ]:
#upload file
from google.colab import files
uploaded = files.upload()

df = pd.read_excel(io.BytesIO(uploaded['My_Recruiting_Jobs_-_More_Details.xlsx']))

In [ ]:
#remove reqs with 0 openings
df = df.loc[df['Number of Openings Available'] != 0]

#remove reqs with 'freeze' or 'filled' in req name
df = df.loc[df['Job Requisition'].str.contains('Freeze Date')==False]
df = df.loc[df['Job Requisition'].str.contains('Fill Date')==False]


In [ ]:
#fix dtypes
# print(df.dtypes)

#remove extra text from start date col
df['Recruiting Start Date'] = df['Recruiting Start Date'].str[:10]
#change to date type
df['Recruiting Start Date'] = pd.to_datetime(df['Recruiting Start Date'], format='%m/%d/%Y')
df['Recruiting Start Date'] = df['Recruiting Start Date'].dt.date

#remove extra text in job requisition column
df['Job Requisition'] = df['Job Requisition'].str[9:-7]

#remove columns
df = df.drop(['Candidates in Review Stage', 'Active Candidates', 'Manager Photo', 'Supervisory Organization', 'Additional Locations',
         'Date Request Entered', 'Request Completed Date', 'Target Hire Date', 'Job Family'], axis=1)
         
#add columns
df['Status'] = ''
df['Days Open'] = np.nan
df['Notes'] =''

#reorganize column order
df = df[['Job Requisition', 'Status', 'Number of Openings Available', 'Recruiting Start Date', 'Days Open',
         'Notes', 'Recruiter', 'Division', 'Cost Center Hierarchy', 'Cost Center',
         'Primary Location', 'Hiring Manager', 'Create Job Requisition Reason', 'Replacement for Worker']]

#rename column names
df = df.rename(columns={'Job Requisition':'Job', 'Number of Openings Available':'Openings', 'Recruiting Start Date':'Open Date', 
                        'Cost Center Hierarchy':'Business Unit', 'Cost Center':'Department', 'Create Job Requisition Reason':'Reason for Hire', 
                        })

In [ ]:
#correct reason for hire text
df['Reason for Hire'].mask(df['Reason for Hire'] == 'Recruiting > Backfill', 'Backfill', inplace=True)
df['Reason for Hire'].mask(df['Reason for Hire'] == 'Recruiting > New Hire', 'New Hire', inplace=True)

#sort
df.sort_values(['Recruiter', 'Division', 'Business Unit', 'Department'], inplace=True)

In [ ]:
df.to_excel('Jobs Summary WD.xlsx', encoding = 'utf-8-sig', index=False) 
files.download('Jobs Summary WD.xlsx')